In [1]:
import Visualization as viz
from ipywidgets import widgets
from IPython.display import display, clear_output
from bokeh.plotting import figure, show, output_notebook

output_notebook()

Loading BokehJS ...

In [2]:
from pymongo import MongoClient

client = MongoClient()
db = client.VentDB
breath_db = db.breath_collection
train_db = db.train_collection

train_db.find().count()

10000

In [3]:
breath_db.find().count()

1964338

In [4]:
def on_button_click(b):
    global breaths

    clear_output()
    if results.alive:
        if breaths != 1:
            viz.update_database(breaths['_id'], w.selected_labels)

        breaths = results.next()
        df, breath_start, breath_end = viz.breath_viz(breaths['_id'])
        p=viz.make_plot(df, breath_start, breath_end)
        show(p)
    else:
        if breaths != 1:
            viz.update_database(breaths['_id'], w.selected_labels)
        clear_output()
        print('No more')

In [5]:
train_db.find({'validation':{'$exists':0}, 'breath_character.exp_time': {'$lte': 1000}}).count()

942

In [6]:
import pandas as pd
val_df = pd.io.json.json_normalize(list(train_db.find({'validation':{'$exists':1}}, {'validation':1})))

In [7]:
val_df.replace({0:False, 1:True}, inplace=True)
val_df = val_df[['validation.ds', 'validation.fa', 'validation.garb',
       'validation.ie', 'validation.norm', 'validation.pds', 'validation.pfl',
       'validation.pvt']].astype(bool)

In [8]:
val_df['validation.ds'].value_counts()

False    1568
True       25
Name: validation.ds, dtype: int64

In [9]:
val_df['validation.pds'].value_counts()

False    1574
True       19
Name: validation.pds, dtype: int64

In [10]:
val_df['validation.pfl'].value_counts()

False    1179
True      414
Name: validation.pfl, dtype: int64

In [11]:
val_df['validation.pvt'].value_counts()

False    1496
True       97
Name: validation.pvt, dtype: int64

In [12]:
val_df['validation.ie'].value_counts()

False    1272
True      321
Name: validation.ie, dtype: int64

In [13]:
val_df['validation.norm'].value_counts()

True     831
False    762
Name: validation.norm, dtype: int64

In [14]:
breaths = 1
results = viz.get_breaths(75)
w = widgets.SelectMultiple(description='Types', options=['Normal', 'DoubleStacked', 'PostDoubleStacked', 
                                                         'PressureFlowLimited', 'FlowAbnormal', 
                                                         'PrematureVentTermination', 'IneffectiveTrigger', 'Garbage'])
w.height = 150
button = widgets.Button(description='Next')
container = widgets.HBox(children=[w, button])

button.on_click(on_button_click)
display(container)

breath error: 661    122
Name: patient_ID, dtype: int64/8/2015-04-18 11:42:12/21152
[{'date_time': 1429378932.0, 'file': '/media/veracrypt1/Research_Data/VentDysSedation/RawData/P122/hm_Waveform 15-04-18-11-41.txt', '_id': '661    122\nName: patient_ID, dtype: int64/8/2015-04-18 11:42:12/21152', 'breath_num': 8}]


# Use PDS breaths to ID DS Breaths

In [ ]:
pds = list(train_db.find({'validation':{'$exists':1}, 'validation.pds':1}, {'file':1, 'breath_num':1}))
for breath in pds:
    data = breath_db.find_one({'file': breath['file'], 'breath_num':breath['breath_num']-1})
    data['validation'] = {'norm':0,
                                'ds': 1,
                                'pds': 0,
                                'pfl': 0,
                                'fa': 0,
                                'pvt':0,
                                'ie': 0,
                                'garb': 0}

    try:
        train_db.insert_one(data)
    except:
        train_db.update_one({'file': breath['file'], 'breath_num':breath['breath_num']-1}, {'$set':data})

In [ ]:
ids = list(train_db.find({'validation':{'$exists':1}}, {'_id':1, 'validation':1}).limit(20))

In [ ]:
for _id in ids:
    print(_id['validation'])
    df, breath_start, breath_end = viz.breath_viz(_id['_id'])
    p=viz.make_plot(df, breath_start, breath_end)
    show(p)